In [13]:
#Importing Libraries

import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import scale
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
### Data preprocessing
train = pd.read_csv("Data/data_train_pre.csv", )
test = pd.read_csv("Data/data_test_pre.csv", )

train[train['scheme_management']=="None"]['scheme_management'] = "unknown"
test[test['scheme_management']=="None"]['scheme_management'] = "unknown"
train=train.drop('extraction_type',axis=1,inplace=False)

test=test.drop('extraction_type',axis=1,inplace=False)
train=train.drop(['Unnamed: 0','date_recorded','quality_group','quantity_group', 'management_group','source_type','management_group','extraction_type_group',
                  'district_code','region_code','payment_type','scheme_name'],axis=1,inplace=False)
test=test.drop(['Unnamed: 0','date_recorded','quality_group','quantity_group', 'management_group','source_type','management_group','extraction_type_group',
                  'district_code','region_code','payment_type','scheme_name'],axis=1,inplace=False)
count=train['status_group'].value_counts()

count

train.columns.values

array(['region', 'amount_tsh', 'funder', 'gps_height', 'installer',
       'longitude', 'latitude', 'wpt_name', 'num_private', 'basin', 'lga',
       'population', 'public_meeting', 'scheme_management', 'permit',
       'extraction_type_class', 'management', 'payment', 'water_quality',
       'quantity', 'source', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'age', 'loc_type', 'employment_rate',
       'popdeath_rate', 'status_group', 'crime_rating'], dtype=object)

In [9]:
# creating train (train_s1) and hold out data set (holdout) and Test (train_s2)
import random
length=len(train)
rows = random.sample(train.index, length/2)
train_s1 = train.ix[rows]
count=train_s1['status_group'].value_counts()
train_s3 = train.drop(rows)
length=len(train_s3)
rows = random.sample(train_s3.index, length/2)
train_s2 = train_s3.ix[rows]
holdout=train_s3.drop(rows)





count
len(train_s2)


14850

14850

In [218]:
# writing the downsampling fucntion
def dnsmpl(xtrain,count,p):
    train=xtrain
    positive=count['functional needs repair']
    negative=int(positive*((1-p)/p))
    train_0=train[train['status_group']<>'functional needs repair']
    train_1=train[train['status_group']=='functional needs repair']
    train_0_down_sampled = train_0.sample( negative)
    train_final=train_1.append(train_0_down_sampled)
    return(train_final)

#test y
y=train_s2['status_group']
y_binary=pd.get_dummies(y)
y_bin=y_binary['functional needs repair']
train_s2_dum=pd.get_dummies(train_s2)

train1=train_s1.drop('status_group',axis=1,inplace=False)



x=dnsmpl(train_s1,count,0.20)
x['status_group'].value_counts()

functional                 5070
non functional             3558
functional needs repair    2157
dtype: int64

# PCA

In [209]:
train1 = train1.drop('region',axis = 1)


In [213]:
### PCA
train_dum=pd.get_dummies(train1)
test_dum=pd.get_dummies(test)
columns=test_dum.columns.values
columns1=train_dum.columns.values
columns2=set(columns).intersection(columns1)
train_dum=train_dum[list(columns2)]
test_dum=test_dum[list(columns2)]
print len(columns2)
train_dum_scale=scale(train_dum)
test_dum_scale=scale(test_dum)
pca = PCA(n_components=100)
pca_fit=pca.fit(train_dum_scale)
train_pca=pca_fit.transform(train_dum_scale)
test_pca=pca_fit.transform(test_dum_scale)
pca.explained_variance_ratio_ 
train_pca=np.matrix(train_pca)
train_pca=pd.DataFrame(train_pca)
test_pca=np.matrix(test_pca)
test_pca=pd.DataFrame(test_pca)

251


In [214]:

train1y=train_s1['status_group']
train1y_bins=pd.get_dummies(train1y)
train1y_binary = train1y_bins['functional needs repair']

In [215]:
train1y_binary[:6]

6187     0
47754    0
47780    1
36973    0
7215     0
51151    1
Name: functional needs repair, dtype: float64

In [227]:
train_s1['region'].values

array(['Iringa', 'Mbeya', 'Mbeya', ..., 'Tabora', 'Shinyanga', 'Shinyanga'], dtype=object)

In [240]:
names = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10','status_group', 'region']
x_names = ['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10', 'region']
#water01train = pd.concat([a['functional needs repair'],train_pca.iloc[:,0:10]], axis = 1, ignore_index = True)
#water01train = pd.merge(a,train_pca.iloc[:,0:10], on= )
train_pca['status_group'] = train1y_binary.values
train_pca['region'] = train_s1['region'].values
test_pca['region'] = train_s2['region'].values
water01train = train_pca.loc[:,[0,1,2,3,4,5,6,7,8,9,'status_group','region']]
water01train.columns = names
water01holdout_x = test_pca.loc[:,[0,1,2,3,4,5,6,7,8,9,'region']]
water01holdout_x.columns = x_names

In [241]:
water01holdout_x[:6]

,pc1,pc2,pc3,pc4,pc5,pc6,pc7,pc8,pc9,pc10,region
0,3.666705,0.268024,1.333375,-0.109285,-0.524446,2.726273,-0.891415,-0.798753,-1.044141,-0.981819,Kigoma
1,3.041574,-0.013271,-1.949048,0.318884,3.985971,-0.346055,-0.729784,-1.147758,0.349691,0.019928,Mbeya
2,3.701646,-0.155036,0.761051,-0.430093,1.309569,1.098686,0.081737,-1.283052,-0.457834,-0.195730,Iringa
3,0.549247,-0.348886,1.888424,0.662245,-0.373116,-0.449559,-2.184206,-0.527851,-0.431154,-0.018507,Mwanza
4,3.199873,0.581616,-1.242400,0.336622,3.860723,-0.463371,-0.882661,-1.133755,-0.271246,-0.090784,Kagera
5,2.811904,-2.070088,5.520357,-4.054523,-1.755904,0.660342,-4.529408,-4.259834,6.117628,-1.201643,Mtwara


In [144]:
?pd.DataFrame

In [231]:
md = smf.mixedlm('status_group ~ pc1', data = water01train, groups = water01train['region']) 
md_fit = md.fit()
print(md_fit.summary())

           Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: status_group
No. Observations: 29700   Method:             REML        
No. Groups:       21      Scale:              0.0651      
Min. group size:  419     Likelihood:         -1629.2041  
Max. group size:  2627    Converged:          Yes         
Mean group size:  1414.3                                  
----------------------------------------------------------
                 Coef. Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept        0.068    0.011  6.457 0.000  0.047  0.089
pc1              0.009    0.001 13.257 0.000  0.008  0.011
Intercept RE     0.002    0.003                           



/Users/julianghadially/anaconda/lib/python2.7/site-packages/statsmodels/regression/mixed_linear_model.py:1717: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [250]:

yhat = md_fit.predict(water01holdout_x[['pc1','region']])

NotImplementedError: 

In [253]:
coefs = md_fit.fe_params
yhat = coefs[0] + coefs[1]*water01holdout_x['pc1']
yhat[:6]

0    0.102747
1    0.096850
2    0.103077
3    0.073339
4    0.098343
5    0.094684
Name: pc1, dtype: float64